In [ ]:
# Instalar dependencias necesarias
!pip install yfinance pmdarima

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [ ]:
# Definir el ticker y descargar datos semanales
ticker = 'AAPL'
data = yf.download(ticker, start='2019-01-01', end='2024-12-31')
data = data['Close'].resample('W').last().dropna()
data.index.freq = 'W'

In [ ]:
# Entrenamiento: 2020-2023 | Test: 2024
train = data.loc['2020-01-01':'2023-12-31']
test = data.loc['2024-01-01':]

In [ ]:
# Ajustar modelo SARIMA automáticamente con pmdarima
model = auto_arima(train, seasonal=True, m=52, trace=True,
                   error_action='ignore', suppress_warnings=True,
                   stepwise=True)

In [ ]:
# Hacer predicciones para el mismo número de pasos que el conjunto test
n_periods = len(test)
forecast = model.predict(n_periods=n_periods)
forecast_index = test.index
forecast_series = pd.Series(forecast, index=forecast_index)

In [ ]:
# Calcular métricas de error
mse = mean_squared_error(test, forecast_series)
mae = mean_absolute_error(test, forecast_series)
rmse = np.sqrt(mse)

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')

In [ ]:
# Graficar datos reales y pronóstico
plt.figure(figsize=(12,6))
plt.plot(data.index[-260:], data[-260:], label='Datos reales')
plt.plot(forecast_index, forecast_series, label='Pronóstico SARIMA', color='red')
plt.title('Pronóstico SARIMA')
plt.xlabel('Fecha')
plt.ylabel('Precio de cierre')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Residuales del modelo
residuals = test - forecast_series
plt.figure(figsize=(10,4))
plt.plot(residuals, marker='o')
plt.title('Residuos del modelo SARIMA')
plt.grid(True)
plt.show()